# Fine-tune MobileNetV2
This notebook will load in some training data that has been downloaded from [Imagenet](http://image-net.org/explore) and use it to train the model to predict if an image is from one of these five categories:
- chair
- couch
- table
- lamp
- bed

Note: A later version of this notebook will use [ImageDataGenerator](https://keras.io/preprocessing/image/) ([addition info](https://machinelearningmastery.com/image-augmentation-deep-learning-keras/)) to augment the training data. This should help to [further reduce the validation accuracy](https://www.learnopencv.com/keras-tutorial-fine-tuning-using-pre-trained-models/).

In [1]:
from datetime import datetime

import matplotlib.pyplot as plt
import numpy as np
from keras import layers, models, optimizers
from keras_applications import mobilenet_v2 
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

from utils.imagenet.get_images import get_imagenet_data

/Users/dave/anaconda/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Load model

In [2]:
# 224 is the default image size for MobileNetV2 (and many other pre-trained models on Keras)
image_size = 224
mobilenet_conv = mobilenet_v2.MobileNetV2(weights='imagenet',
                                          include_top=False,
                                          input_shape=(image_size, image_size, 3))

## Load data

In [3]:
# Specify types of images to download
download_info = {
    # Imagenet_ID: type_of_image 
    'n03325403': 'chair',
    'n03015149': 'couch',
    'n04379243': 'table',
    'n03636649': 'lamp',
    'n03225988': 'bed'
}

In [6]:
# Download, process, and load the images
processed_images, images_class_softmax = get_imagenet_data(
    download=False,
    download_info=download_info,
    directory='../downloaded_images/fine_tuning',
    images_per_type=500,
    image_size=image_size,
    process=True,
    model=mobilenet_v2
)

TypeError: cannot identify image file '../downloaded_images/fine_tuning/lamp/7fe710dcb617bc68537cf5906469add2441eac38.jpg'
TypeError: cannot identify image file '../downloaded_images/fine_tuning/lamp/a7c4638420d81cb0aaac25c401545e09c9d1c245.jpg'
Mapping:
{'bed': 0, 'chair': 1, 'couch': 2, 'lamp': 3, 'table': 4}


## Create our fine-tuned model
Since we did not include the 'top' (the layer that predicts the type/class of image) of the model when loading it, we need to add this layer to the model. 

In [7]:
N_CATEGORIES = len(download_info.values())

# Create model (currently empty)
model = models.Sequential()
# Add MobileNetV2
model.add(mobilenet_conv)
# Add global pooling layer, which was also left out of the 'top'
model.add(layers.GlobalAveragePooling2D())
# Add dropout to reduce overfitting
model.add(layers.Dropout(0.5))
# Add layer to predict class
model.add(layers.Dense(N_CATEGORIES, activation='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_224 (Model) (None, 7, 7, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d_1 ( (None, 1280)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1280)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 6405      
Total params: 2,264,389
Trainable params: 2,230,277
Non-trainable params: 34,112
_________________________________________________________________


## Prepare data for model

In [8]:
x_train, x_test, y_train, y_test = train_test_split(processed_images,
                                                    images_class_softmax,
                                                    test_size=0.2,
                                                    stratify=images_class_softmax,
                                                    random_state=2)

x_train.shape, len(y_train)

((1857, 224, 224, 3), 1857)

## Compile the model
Define the [key features](https://stackoverflow.com/questions/47995324/does-model-compile-initialize-all-the-weights-and-biases-in-keras-tensorflow) of the model.

In [9]:
# Compile the model
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])

## Train the model
In the interest of saving time and not needing greater results at the moment, the model is only being trained for three epochs and with 1000 images. Both of these numbers are very small and I expect the validation accurary to greatly improve when we increase these values. However, using a GPU, rather than my laptop, would be much more practical for this next step.

Given the simplicity of this work, I'm rather pleased that the validation accuracy is almost 90%. Another thing that I would change for a more formal training of the model is removing sub-par images from the dataset. Some images include humans, pairs of objects, or are poorly focused. This cleaning of the data will indoubtably further improve the results.

In [10]:
model.fit(x_train[:1000],
          y_train[:1000],
          batch_size=32,
          epochs=3,
          validation_data=[x_test, y_test])

Train on 1000 samples, validate on 465 samples
Epoch 1/3
1000/1000 [==============================] - 636s 636ms/step - loss: 1.1108 - acc: 0.5980 - val_loss: 0.5036 - val_acc: 0.8624
Epoch 2/3
1000/1000 [==============================] - 667s 667ms/step - loss: 0.3637 - acc: 0.8900 - val_loss: 0.3781 - val_acc: 0.8925
Epoch 3/3
1000/1000 [==============================] - 675s 675ms/step - loss: 0.1760 - acc: 0.9460 - val_loss: 0.3303 - val_acc: 0.8925


## Save the model

In [11]:
date_today = datetime.today().strftime('%Y_%m_%d')
model_name = 'mobilenet_v2'
accuracy = str(round(model.history.history['val_acc'][-1], 4)).split('.')[1]

save_string = date_today + '_' + model_name + '_' + accuracy
save_string

'2018_07_29_mobilenet_v2_8925'

In [13]:
model.save('saved_models/{}.h5'.format(save_string))